In [ ]:
!pip install -qU langchain langchain-community grandalfa faiss-cpu sentence-transformers pdfplumber requests arabic-reshaper python-bidi

In [12]:
!pip install -qU langchain langchain-community faiss-cpu sentence-transformers pdfplumber requests arabic-reshaper python-bidi

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 54.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 71.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 470.2/470.2 kB 31.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 120.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.8/64.8 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 292.9/292.9 kB 24.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 87.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8

In [13]:
!pip install -qU google-generativeai

In [14]:
from langchain_community.document_loaders import PyPDFDirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings
import requests
import io
import pdfplumber

from IPython.display import display, Markdown
#from IPython.display import displayو Markdown
import os
from bidi.algorithm import get_display
import arabic_reshaper
from pathlib import Path

In [ ]:
urls = []
with open("links.txt", 'r', encoding='utf-8') as file:
    for line in file:
        url = line.strip()
        if url:
            urls.append(url)

def rtl_text(text):
    reshaped_text = arabic_reshaper.reshape(text)
    return get_display(reshaped_text)

if not os.path.exists('pdf_results'):
    os.makedirs('pdf_results')
if not os.path.exists('text_results'):
    os.makedirs('text_results')

for url in urls:

    response = requests.get(url, timeout=10)
    if response.status_code != 200:
        print(f"خطا در دانلود {url} - کد خطا: {response.status_code}")

    pdf_name = url.split('/')[-1].replace('.aspx', '.pdf')
    pdf_path = os.path.join('pdf_results', pdf_name)
    with open(pdf_path, 'wb') as f:
        f.write(response.content)

    with pdfplumber.open(pdf_path) as pdf:
        text = ""
        for page in pdf.pages:
            page_text = page.extract_text() or ""
            text += page_text + "\n"

    txt_name = pdf_name.replace('.pdf', '.txt')
    txt_path = os.path.join('text_results', txt_name)
    with open(txt_path, 'w', encoding='utf-8') as f:
        f.write(text)


In [15]:
pip install pypdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.2/313.2 kB 7.1 MB/s eta 0:00:00


In [28]:
pdf_folder_path = "/content/pdf_results"
loader = PyPDFDirectoryLoader(pdf_folder_path)
docs = loader.load()

In [29]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200
)
text_chunks = text_splitter.split_documents(docs)

embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
db = FAISS.from_documents(text_chunks, embeddings)

retriever = db.as_retriever()
retriever = db.as_retriever(search_kwargs={"k": 4})

In [30]:
!pip install langchain openai pypdf

In [32]:
!pip install -U langchain-openai langchain_core

  Using cached langchain_openai-0.3.28-py3-none-any.whl.metadata (2.3 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.6/70.6 kB 3.4 MB/s eta 0:00:00


In [37]:
from langchain_core.runnables import RunnableParallel, RunnablePassthrough
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import OpenAI
from operator import itemgetter
from getpass import getpass
api_key = getpass("کلید API خود را وارد کنید (متن پنهان می‌ماند): ")

# تعریف PromptTemplate
prompt = PromptTemplate.from_template(
    """پاسخگوی سؤال زیر باشید با استفاده از اطلاعات زمینه‌ای:
    زمینه: {context}
    سؤال: {question}
    پاسخ:"""
)

llm = OpenAI(api_key=api_key, temperature=0)
output_parser = StrOutputParser()

RAG = RunnableParallel(
    {
        "context": itemgetter("question") | retriever,
        "question": RunnablePassthrough(),
    }
)

chain = RAG | prompt | llm | output_parser


کلید API خود را وارد کنید (متن پنهان می‌ماند): ··········


In [38]:
input_question = {"question": "هدف اصلی آیین‌نامه‌های بانک مرکزی چیست؟"}
response = chain.invoke(input_question)
display(Markdown(response))
print(response)

 هدف اصلی آیین‌نامه‌های بانک مرکزی، تنظیم و نظارت بر عملکرد بانک‌ها و مؤسسات مالی است تا از پایداری و امنیت سیستم بانکی و اعتماد عمومی به آن حفظ شود.

 هدف اصلی آیین‌نامه‌های بانک مرکزی، تنظیم و نظارت بر عملکرد بانک‌ها و مؤسسات مالی است تا از پایداری و امنیت سیستم بانکی و اعتماد عمومی به آن حفظ شود.


In [41]:
input_question = {"question": "تسهیلات قرض الحسنه فرزندآوری به چه شرح است؟"}
response = chain.invoke(input_question)
display(Markdown(response))
print(response)

 تسهیلات قرض الحسنه فرزندآوری به معنای اعطای وام به زوجین جهت تسهیل در فرآیند فرزندآوری و تامین هزینه‌های مربوط به آن می‌باشد. این تسهیلات به صورت قرض الحسنه و با شرایط ویژه‌ای از سوی بانک‌ها ارائه می‌شود.

 تسهیلات قرض الحسنه فرزندآوری به معنای اعطای وام به زوجین جهت تسهیل در فرآیند فرزندآوری و تامین هزینه‌های مربوط به آن می‌باشد. این تسهیلات به صورت قرض الحسنه و با شرایط ویژه‌ای از سوی بانک‌ها ارائه می‌شود.
